In [ ]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec
import torch
import matplotlib.pyplot as plt
import re

random.seed(123)
torch.manual_seed(123)
np.random.seed(123)

os.chdir('../models/')
from CoreSNN import *
os.chdir('../tsa/')
from ExplanationCreation import *
from ExplanationEvaluation import *

In [ ]:
dataset = load_obj('data/dataset_max.pkl')
A_testset_t = load_obj('data/quantitative_test_t_A_final.pkl')
B_testset_t = load_obj('data/quantitative_test_t_B_final.pkl')
A_y_true = dataset['y_test_A'][:, A_testset_t]
B_y_true = dataset['y_test_B'][:, B_testset_t]

# Output-completeness

## Read result data

In [ ]:
os.chdir('') #Add path to results
epsilons = {'s': [], 'ns':[], 'sam':[]} # Fill tested epsilon values here

def get_scores(nb_layer, expl_type, epsilons):
    _, y_pred_A_0, y_pred_p_A_0 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_A_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[0]))
    _, y_pred_A_25, y_pred_p_A_25 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_A_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[1]))
    _, y_pred_A_50, y_pred_p_A_50 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_A_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[2]))
    _, y_pred_A_75, y_pred_p_A_75 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_A_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[3]))
    _, y_pred_B_0, y_pred_p_B_0 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_B_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[0]))
    _, y_pred_B_25, y_pred_p_B_25 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_B_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[1]))
    _, y_pred_B_50, y_pred_p_B_50 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_B_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[2]))
    _, y_pred_B_75, y_pred_p_B_75 = load_obj(os.getcwd() + '\\evaluation\\output_completeness\\{}\\{}L_oc_B_epsilon{}.pkl'.format(expl_type, nb_layer, epsilons[3]))

    scores = [balanced_accuracy_score([*y_pred_A_0, *y_pred_B_0], [*y_pred_p_A_0, *y_pred_p_B_0]),
              balanced_accuracy_score([*y_pred_A_25, *y_pred_B_25], [*y_pred_p_A_25, *y_pred_p_B_25]),
              balanced_accuracy_score([*y_pred_A_50, *y_pred_B_50], [*y_pred_p_A_50, *y_pred_p_B_50]),
              balanced_accuracy_score([*y_pred_A_75, *y_pred_B_75], [*y_pred_p_A_75, *y_pred_p_B_75])]
    return scores

In [ ]:
oc_s = []
oc_ns = []
oc_sam = []

for nb_layer in range(3):
    oc_s.append(get_scores(nb_layer, 's', epsilons['s']))
    oc_ns.append(get_scores(nb_layer, 'ns', epsilons['ns']))
    oc_sam.append(get_scores(nb_layer, 'sam', epsilons['sam']))

## Visualize results

In [ ]:
fig = plt.figure(tight_layout=True, frameon=False, figsize=(15,5),dpi=200)
gs = gridspec.GridSpec(1,3)

ax1 = fig.add_subplot(gs[0,0])
ax1.plot(oc_s[0])
ax1.plot(oc_ns[0])
ax1.plot(oc_sam[0], linestyle='dotted')
ax1.set_ylim(ymin = 0, ymax=1)
ax1.set_ylabel('Output-completeness score', fontdict={'fontsize': 16})
ax1.set_xticks(range(4))
ax1.set_xticklabels([0, '25%', '50%', '75%'])
ax1.set_xlabel('Epsilon', fontdict={'fontsize': 16})
ax1.set_title('Output-completeness for explanations of SNN-1L', pad=10)
ax1.legend(['TSA-S','TSA-NS', 'SAM'], prop={'size':13})

ax2 = fig.add_subplot(gs[0,1])
ax2.plot(oc_s[1])
ax2.plot(oc_ns[1])
ax2.plot(oc_sam[1], linestyle='dotted')
ax2.set_ylim(ymin = 0, ymax=1)
ax2.set_ylabel('Output-completeness score', fontdict={'fontsize': 16})
ax2.set_xticks(range(4))
ax2.set_xticklabels([0, '25%', '50%', '75%'])
ax2.set_xlabel('Epsilon', fontdict={'fontsize': 16})
ax2.set_title('Output-completeness for explanations of SNN-2L', pad=10)
ax2.legend(['TSA-S','TSA-NS', 'SAM'], prop={'size':13})

ax3 = fig.add_subplot(gs[0,2])
ax3.plot(oc_s[2])
ax3.plot(oc_ns[2])
ax3.plot(oc_sam[2], linestyle='dotted')
ax3.set_ylim(ymin = 0, ymax=1)
ax3.set_ylabel('Output-completeness score', fontdict={'fontsize': 16})
ax3.set_xticks(range(4))
ax3.set_xticklabels([0, '25%', '50%', '75%'])
ax3.set_xlabel('Epsilon', fontdict={'fontsize': 16})
ax3.set_title('Output-completeness for explanations of SNN-3L', pad=10)
ax3.legend(['TSA-S','TSA-NS', 'SAM'], prop={'size':13})

# Faithfulness

## Reading the data

In [ ]:
# Perhaps adjust the paths

y_preds_p_oneA_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_onelayer_A.pkl')
y_preds_p_oneB_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_onelayer_B.pkl')
y_preds_p_twoA_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_twolayer_A.pkl')
y_preds_p_twoB_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_twolayer_B.pkl')
y_preds_p_threeA_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_threelayer_A.pkl')
y_preds_p_threeB_ns = load_obj(os.getcwd()+'/faithfulness/tsa-ns/y_preds_perturbed_threelayer_B.pkl')

y_preds_p_oneA_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_onelayer_A.pkl')
y_preds_p_oneB_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_onelayer_B.pkl')
y_preds_p_twoA_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_twolayer_A.pkl')
y_preds_p_twoB_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_twolayer_B.pkl')
y_preds_p_threeA_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_threelayer_A.pkl')
y_preds_p_threeB_s = load_obj(os.getcwd()+'/faithfulness/tsa-s/y_preds_perturbed_threelayer_B.pkl')

y_preds_p_oneA_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_onelayer_A.pkl')
y_preds_p_oneB_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_onelayer_B.pkl')
y_preds_p_twoA_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_twolayer_A.pkl')
y_preds_p_twoB_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_twolayer_B.pkl')
y_preds_p_threeA_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_threelayer_A.pkl')
y_preds_p_threeB_baseline = load_obj(os.getcwd()+'/faithfulness/baseline_y_preds_perturbed_threelayer_B.pkl')

In [ ]:
# no feature segments found for A t=9 in the explanations extracted from ThreeLayerSNN, so only consider the original prediction 
y_preds_p_threeA_s[37] = [5]

## Normalization and combination of the results per model

In [ ]:
def get_perf_curve_yhat(y_preds_p_A, y_preds_p_B):
    y_hat = [pred[0] for pred in y_preds_p_A]
    y_hat_B = [pred[0] for pred in y_preds_p_B]
    y_hat.extend(y_hat_B)

    perf = []
    for i in range(max([len(y_pred) for y_pred in y_preds_p_A])):
        y_pred_p = [pred[i] if i<len(pred) else pred[-1] for pred in y_preds_p_A]
        y_pred_p_B = [pred[i] if i<len(pred) else pred[-1] for pred in y_preds_p_B]
        y_pred_p.extend(y_pred_p_B)
        perf.append(balanced_accuracy_score(y_hat, y_pred_p))
    return perf

def get_perf_curve_ytrue(y_preds_p_A, y_preds_p_B):
    perf = []
    for i in range(max([len(y_pred) for y_pred in y_preds_p_A])):
        y_pred_p = [pred[i] if i<len(pred) else pred[-1] for pred in y_preds_p_A]
        y_pred_p_B = [pred[i] if i<len(pred) else pred[-1] for pred in y_preds_p_B]
        y_pred_p.extend(y_pred_p_B)
        perf.append(balanced_accuracy_score([*A_y_true[0], *B_y_true[0]], y_pred_p))
    return perf

In [ ]:
perf_one_yhat_ns = get_perf_curve_yhat(y_preds_p_oneA_ns, y_preds_p_oneB_ns)
perf_one_ytrue_ns = get_perf_curve_ytrue(y_preds_p_oneA_ns, y_preds_p_oneB_ns)
perf_one_yhat_s = get_perf_curve_yhat(y_preds_p_oneA_s, y_preds_p_oneB_s)
perf_one_ytrue_s = get_perf_curve_ytrue(y_preds_p_oneA_s, y_preds_p_oneB_s)
perf_one_yhat_baseline = get_perf_curve_yhat(y_preds_p_oneA_baseline, y_preds_p_oneB_baseline)
perf_one_ytrue_baseline = get_perf_curve_ytrue(y_preds_p_oneA_baseline, y_preds_p_oneB_baseline)

perf_two_yhat_ns = get_perf_curve_yhat(y_preds_p_twoA_ns, y_preds_p_twoB_ns)
perf_two_ytrue_ns = get_perf_curve_ytrue(y_preds_p_twoA_ns, y_preds_p_twoB_ns)
perf_two_yhat_s = get_perf_curve_yhat(y_preds_p_twoA_s, y_preds_p_twoB_s)
perf_two_ytrue_s = get_perf_curve_ytrue(y_preds_p_twoA_s, y_preds_p_twoB_s)
perf_two_yhat_baseline = get_perf_curve_yhat(y_preds_p_twoA_baseline, y_preds_p_twoB_baseline)
perf_two_ytrue_baseline = get_perf_curve_ytrue(y_preds_p_oneA_baseline, y_preds_p_oneB_baseline)

perf_three_yhat_ns = get_perf_curve_yhat(y_preds_p_threeA_ns, y_preds_p_threeB_ns)
perf_three_ytrue_ns = get_perf_curve_ytrue(y_preds_p_threeA_ns, y_preds_p_threeB_ns)
perf_three_yhat_s = get_perf_curve_yhat(y_preds_p_threeA_s, y_preds_p_threeB_s)
perf_three_ytrue_s = get_perf_curve_ytrue(y_preds_p_threeA_s, y_preds_p_threeB_s)
perf_three_yhat_baseline = get_perf_curve_yhat(y_preds_p_threeA_baseline, y_preds_p_threeB_baseline)
perf_three_ytrue_baseline = get_perf_curve_ytrue(y_preds_p_oneA_baseline, y_preds_p_oneB_baseline)


In [ ]:
ess_yhat_one_ns = metrics.auc(range(len(perf_one_yhat_ns)), perf_one_yhat_ns)
norm_ess_yhat_one_ns = ess_yhat_one_ns/len(perf_one_yhat_ns)
ess_yhat_one_s = metrics.auc(range(len(perf_one_yhat_s)), perf_one_yhat_s)
norm_ess_yhat_one_s = ess_yhat_one_s/len(perf_one_yhat_s)
ess_yhat_one_baseline = metrics.auc(range(len(perf_one_yhat_baseline)), perf_one_yhat_baseline)
norm_ess_yhat_one_baseline = ess_yhat_one_baseline/len(perf_one_yhat_baseline)

ess_yhat_two_ns = metrics.auc(range(len(perf_two_yhat_ns)), perf_two_yhat_ns)
norm_ess_yhat_two_ns = ess_yhat_two_ns/len(perf_two_yhat_ns)
ess_yhat_two_s = metrics.auc(range(len(perf_two_yhat_s)), perf_two_yhat_s)
norm_ess_yhat_two_s = ess_yhat_two_s/len(perf_two_yhat_s)
ess_yhat_two_baseline = metrics.auc(range(len(perf_two_yhat_baseline)), perf_two_yhat_baseline)
norm_ess_yhat_two_baseline = ess_yhat_two_baseline/len(perf_two_yhat_baseline)

ess_yhat_three_ns = metrics.auc(range(len(perf_three_yhat_ns)), perf_three_yhat_ns)
norm_ess_yhat_three_ns = ess_yhat_three_ns/len(perf_three_yhat_ns)
ess_yhat_three_s = metrics.auc(range(len(perf_three_yhat_s)), perf_three_yhat_s)
norm_ess_yhat_three_s = ess_yhat_three_s/len(perf_three_yhat_s)
ess_yhat_three_baseline = metrics.auc(range(len(perf_three_yhat_baseline)), perf_three_yhat_baseline)
norm_ess_yhat_three_baseline = ess_yhat_three_baseline/len(perf_three_yhat_baseline)

print('ESS for TSA-NS Explanations from OneLayerSNN: ', norm_ess_yhat_one_ns, ' +- ', conf_interval(norm_ess_yhat_one_ns, 180))
print('ESS for TSA-S Explanations from OneLayerSNN: ', norm_ess_yhat_one_s, ' +- ', conf_interval(norm_ess_yhat_one_s, 180))
print('ESS for Baseline Explanations from OneLayerSNN: ', norm_ess_yhat_one_baseline, ' +- ', conf_interval(norm_ess_yhat_one_baseline, 180))
print('ESS for TSA-NS Explanations from TwoLayerSNN: ', norm_ess_yhat_two_ns, ' +- ', conf_interval(norm_ess_yhat_two_ns, 180))
print('ESS for TSA-S Explanations from TwoLayerSNN: ', norm_ess_yhat_two_s, ' +- ', conf_interval(norm_ess_yhat_two_s, 180))
print('ESS for Baseline Explanations from TwoLayerSNN: ', norm_ess_yhat_two_baseline, ' +- ', conf_interval(norm_ess_yhat_two_baseline, 180))
print('ESS for TSA-NS Explanations from ThreeLayerSNN: ', norm_ess_yhat_three_ns, ' +- ', conf_interval(norm_ess_yhat_three_ns, 180))
print('ESS for TSA-S Explanations from ThreeLayerSNN: ', norm_ess_yhat_three_s, ' +- ', conf_interval(norm_ess_yhat_three_s, 180))
print('ESS for Baseline Explanations from ThreeLayerSNN: ', norm_ess_yhat_three_baseline, ' +- ', conf_interval(norm_ess_yhat_three_baseline, 180))

In [ ]:
(norm_ess_yhat_one_baseline + norm_ess_yhat_two_baseline + norm_ess_yhat_three_baseline)/3

In [ ]:
fig = plt.figure(tight_layout=True, dpi=150, frameon=False, figsize=(15,5))
gs = gridspec.GridSpec(1,3)

ax1 = fig.add_subplot(gs[0,0])
ax1.plot(perf_one_yhat_s, color=(194/256, 154/256, 177/256))
ax1.plot(perf_one_yhat_ns, color=(219/256, 154/256, 143/256))
ax1.plot(perf_one_yhat_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax1.set_ylim(0,1)
ax1.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax1.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax1.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})
ax1.set_title('Balanced accuracy of OneLayerSNN\n with flipped feature segments')

ax2 = fig.add_subplot(gs[0,1])
ax2.plot(perf_two_yhat_s, color=(194/256, 154/256, 177/256))
ax2.plot(perf_two_yhat_ns, color=(219/256, 154/256, 143/256))
ax2.plot(perf_two_yhat_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax2.set_ylim(0,1)
ax2.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax2.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax2.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})
ax2.set_title('Balanced accuracy of TwoLayerSNN \n with flipped feature segments')

ax3 = fig.add_subplot(gs[0,2])
ax3.plot(perf_three_yhat_s, color=(194/256, 154/256, 177/256))
ax3.plot(perf_three_yhat_ns, color=(219/256, 154/256, 143/256))
ax3.plot(perf_three_yhat_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax3.set_ylim(0,1)
ax3.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax3.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax3.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})
ax3.set_title('Balanced accuracy of ThreeLayerSNN \n with flipped feature segments')



In [ ]:
fig = plt.figure(tight_layout=True, dpi=150, frameon=False, figsize=(15,5))
gs = gridspec.GridSpec(1,3)

ax1 = fig.add_subplot(gs[0,0])
ax1.plot(perf_one_ytrue_s, color=(194/256, 154/256, 177/256))
ax1.plot(perf_one_ytrue_ns, color=(219/256, 154/256, 143/256))
ax1.plot(perf_one_ytrue_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax1.set_ylim(0,1)
ax1.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax1.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax1.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})

ax2 = fig.add_subplot(gs[0,1])
ax2.plot(perf_two_ytrue_s, color=(194/256, 154/256, 177/256))
ax2.plot(perf_two_ytrue_ns, color=(219/256, 154/256, 143/256))
ax2.plot(perf_two_ytrue_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax2.set_ylim(0,1)
ax2.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax2.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax2.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})

ax3 = fig.add_subplot(gs[0,2])
ax3.plot(perf_one_ytrue_s, color=(194/256, 154/256, 177/256))
ax3.plot(perf_three_ytrue_ns, color=(219/256, 154/256, 143/256))
ax3.plot(perf_three_ytrue_baseline, color=(115/256, 75/256, 103/256), linestyle='dotted')
ax3.set_ylim(0,1)
ax3.set_ylabel('Balanced accuracy', fontdict={'size': 16})
ax3.set_xlabel('Number of flipped segments', fontdict={'size': 16})
ax3.legend(['TSA-S', 'TSA-NS', 'Baseline'], prop={'size':13})



# Sensitivity

### Read results

In [ ]:
max_sensitivity_s_oneA = load_obj('/sensitivity/tsa-s/max_sensitivity_oneA.pkl')
max_sensitivity_s_oneB = load_obj('/sensitivity/tsa-s/max_sensitivity_oneB.pkl')
max_sensitivity_s_one = max(max_sensitivity_s_oneA, max_sensitivity_s_oneB)

max_sensitivity_s_twoA = load_obj('/sensitivity/tsa-s/max_sensitivity_twoA.pkl')
max_sensitivity_s_twoB = load_obj('/sensitivity/tsa-s/max_sensitivity_twoB.pkl')
max_sensitivity_s_two = max(max_sensitivity_s_twoA, max_sensitivity_s_twoB)

max_sensitivity_s_threeA = load_obj('/sensitivity/tsa-s/max_sensitivity_threeA.pkl')
max_sensitivity_s_threeB = load_obj('/sensitivity/tsa-s/max_sensitivity_threeB.pkl')
max_sensitivity_s_three = max(max_sensitivity_s_threeA, max_sensitivity_s_threeB)

max_sensitivity_ns_oneA = load_obj('/sensitivity/tsa-ns/max_sensitivity_oneA.pkl')
max_sensitivity_ns_oneB = load_obj('/sensitivity/tsa-ns/max_sensitivity_oneB.pkl')
max_sensitivity_ns_one = max(max_sensitivity_ns_oneA, max_sensitivity_ns_oneB)

max_sensitivity_ns_twoA = load_obj('/sensitivity/tsa-ns/max_sensitivity_twoA.pkl')
max_sensitivity_ns_twoB = load_obj('/sensitivity/tsa-ns/max_sensitivity_twoB.pkl')
max_sensitivity_ns_two = max(max_sensitivity_ns_twoA, max_sensitivity_ns_twoB)

max_sensitivity_ns_threeA = load_obj('/sensitivity/tsa-ns/max_sensitivity_threeA.pkl')
max_sensitivity_ns_threeB = load_obj('/sensitivity/tsa-ns/max_sensitivity_threeB.pkl')
max_sensitivity_ns_three = max(max_sensitivity_ns_threeA, max_sensitivity_ns_threeB)

baseline_sensitivity_A = load_obj('/sensitivity/max_sensitivity_baseline_A.pkl')
baseline_sensitivity_B = load_obj('/sensitivity/max_sensitivity_baseline_B.pkl')
max_sensitivity_baseline = max(baseline_sensitivity_A, baseline_sensitivity_B)

### Analyze

In [ ]:
sensitivities_s = [max_sensitivity_s_one, max_sensitivity_s_two, max_sensitivity_s_three]
sensitivities_ns = [max_sensitivity_ns_one, max_sensitivity_ns_two, max_sensitivity_ns_three]
df_sensitivity = pd.DataFrame([sensitivities_s, sensitivities_ns]).transpose()

In [ ]:
df_sensitivity

In [ ]:
max_sensitivity_baseline